In [1]:
from IPython.core.display import display, HTML
from functools import partial
from functools import cache

MONTHS = ['January', 'Febuary', 'March', 
          'April', 'May', 'June',
          'July', 'August', 'September', 
          'October', 'November', 'December']
MONTHS_IDX = {month: idx for idx, month in enumerate(MONTHS)}

def parse_month(date):
    return date.split(" ")[0]

def parse_day(date):
    return date.split(" ")[1]

def create_grid_list(dates, pruned={}):
    "Turn the set of dates into a list to populate for the display grid."
    months = sorted(list(set(parse_month(date) for date in dates)), 
                    key=lambda month: MONTHS_IDX[month])
    days = sorted(list(set(parse_day(date) for date in dates)),
              key=lambda day: int(day))

    grid_list = []
    for m in months:
        grid_list.append(str(m))
        for d in days:
            test_date = m + " " + d
            if test_date in dates:
                if test_date in pruned:
                    grid_list.append('-')
                else:
                    grid_list.append("O")
            else:
                grid_list.append(" ")

    return grid_list, months, days

def show(dates, pruned={}):
    grid_list, months, days = create_grid_list(dates, pruned)
    num_months = len(months)
    num_days = len(days)
#     print(([''] + days))
    CSS = '''<style>
        .dStyle td {width:2em; height:2em; border:1px solid grey; text-align:center}
        .dStyle td:first-child {border-left:solid}
        .dStyle td:nth-child(1) {border-right:solid}
        .dStyle td:last-child {border-right:solid}
        .dStyle tr:first-child  {border-top:solid}
        .dStyle tr:nth-child(1) {border-bottom:solid}
        .dStyle tr:last-child {border-bottom:solid}
        </style>'''
    table = ('<table class="dStyle">' + '<tr>' + (num_days+1) * '<td>{}').format(*([''] + days))
    table += num_months * ('<tr>' + (num_days+1) * '<td>{}') + '</table>'
    display(HTML(CSS + table.format(*grid_list)))
    


In [2]:
cheryl_dates = {
    'May 15',    'May 16',    'May 19',
    'June 17',   'June 18',
    'July 14',   'July 16',
    'August 14', 'August 15', 'August 17'}

pruned = {'August 14', 'July 16'}
show(cheryl_dates, pruned)

In [3]:
def tell(part):
    "Cheryl tells a part of her birthdate; return a subset of DATES that match the part."
    return {date for date in DATES if part in date}

def know(possible_dates):
    "A person knows the birthdate if they know there is exactly one possible date."
    return len(possible_dates) == 1

def hear(possible_dates, *statements):
    "Return the subset of possible dates that are consistent with all the statements."
    return {date for date in possible_dates
            if all(stmt(date) for stmt in statements)}

def using(dates):
    "Make dates be the value of the global variable DATES."
    global DATES # This is necessary because `tell` looks at `DATES`
    DATES = dates
    return dates

def show_with_statements(dates, statements):
    using(dates)
    return_set = hear(dates, *statements)
    show(dates, dates - 
         hear(dates, *statements))
    return return_set

def show_progression(dates, statements):
    using(dates)
    for i in range(len(statements) + 1):
        if i > 0: 
            print("\n\n")
            if type(statements[i-1]) is partial:
                print(statements[i-1].func.__doc__)
            else:
                print(statements[i-1].__doc__)
        return_set = show_with_statements(dates, statements[:i])
    return return_set

In [4]:
def albert1(date):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_being_told = tell(parse_month(date))
    return (not know(after_being_told) 
            and all(not know(tell(parse_day(d)))
                    for d in after_being_told))

def bernard1(date):
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    at_first = tell(parse_day(date))
    return (not know(at_first)
            and know(hear(at_first, albert1)))

def albert2(date):
    "Albert: Then I also know when Cheryl's birthday is."
    return know(hear(tell(parse_month(date)), bernard1))

In [5]:
cheryl_statements = [albert1, bernard1, albert2]
dates = cheryl_dates
show_progression(dates, cheryl_statements)




Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too.





Bernard: At first I don't know when Cheryl's birthday is, but I know now.





Albert: Then I also know when Cheryl's birthday is.


{'July 16'}

In [6]:
INTERESTING_DATES = {'June 26', 'June 13', 'April 19', 'May 14', 'March 27', 'July 16', 'March 25', 'June 24', 'April 24', 'May 27', 'June 25', 'April 29', 'March 26', 'July 14', 'March 24'}
dates = INTERESTING_DATES
show_progression(dates, cheryl_statements)




Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too.





Bernard: At first I don't know when Cheryl's birthday is, but I know now.





Albert: Then I also know when Cheryl's birthday is.


{'May 14'}

In [7]:
import random
from tqdm.notebook import tqdm

def gen_dates(k=10):
    "Pick a set of dates"
    some_dates = {mo + ' ' + d1 + d2
              for mo in ('March', 'April', 'May', 'June', 'July')
              for d1 in '12'
              for d2 in '3456789'}

    return set(random.sample(tuple(some_dates), k))

def gen_solution(statements, k=10, max_attempts=10000):
    for _ in tqdm(range(max_attempts)):
        dates = gen_dates(k)
        using(dates)
        if hear(dates, *statements[:len(statements)//2]):
            solution = hear(dates, *statements)
            if know(solution):
                return dates, solution
    return None, None

In [8]:
tmp_dates, solution = gen_solution(cheryl_statements)
print(tmp_dates)
# show_progression(tmp_dates, cheryl_statements)


{'June 23', 'June 25', 'March 26', 'June 15', 'March 17', 'April 25', 'July 23', 'July 15', 'May 28', 'March 13'}


In [53]:
def albert1(date):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_being_told = tell(parse_month(date))
    return (not know(after_being_told) 
            and all(not know(tell(parse_day(d)))
                    for d in after_being_told))

def bernard1(date):
    "Bernard: I still don't know"
    return not know(hear(tell(parse_day(date)), albert1))

def albert2(date):
    "Albert: I still don't know."
    return not know(hear(tell(parse_month(date)), bernard1))

def bernard2(date):
    "Bernard: I still don't know 2"
    return not know(hear(tell(parse_day(date)), albert2))

def bernard3(date):
    "Bernard: OK, now I know."
    return know(hear(tell(parse_day(date)), albert3))

def albert3(date):
    "Albert: OK, now I know."
    return know(hear(tell(parse_month(date)), bernard2))


def month_initial(date):
    "month: I don't know when Cheryl's birthday is, but I know that month does not know too."
    after_being_told = tell(parse_month(date))
    return (not know(after_being_told) 
            and all(not know(tell(parse_day(d)))
                    for d in after_being_told))

def month_initial_simple(date):
    "month: I don't know when Cheryl's birthday is."
    return not know(tell(parse_month(date)))

def day_not_know_complicated(date, prior_statement):
    "day: I still don't know, but I know that month does not know."
    after_hear_day = hear(tell(parse_day(date)), prior_statement)
    after_hear_month = hear(tell(parse_month(date)), prior_statement)
    return (not know(after_hear_day)
            and all(not know(after_hear_month)
                    for d in after_hear_day))

def month_not_know_complicated(date, prior_statement):
    "month: I still don't know, but I know that day does not know."
    after_hear_day = hear(tell(parse_day(date)), prior_statement)
    after_hear_month = hear(tell(parse_month(date)), prior_statement)
    return (not know(after_hear_month)
            and all(not know(after_hear_day)
                    for d in after_hear_month))


@cache
def day_not_know(date, prior_statement):
    "Day: That helps, but I still don't know."
    # Same as day not know, but using for the doc string.
    return not know(hear(tell(parse_day(date)), prior_statement))        


@cache
def month_not_know(date, prior_statement):
    "month: That helps, but I still don't know."
    # Same as day not know, but using for the doc string.
    return not know(hear(tell(parse_month(date)), prior_statement))

# def day_not_know(date, prior_statement):
#     "month: I still don't know, but I know that day does not know."
#     # Same as day not know, but using for the doc string.
#     after_hear_day = hear(tell(parse_day(date)), prior_statement)
#     after_hear_month = hear(tell(parse_month(date)), prior_statement)
#     if len(after_hear_month - after_hear_day) > 0:
#         return not know(after_hear_day)
#     return False

# # @cache
# def month_not_know(date, prior_statement):
#     "month: I still don't know, but I know that day does not know."
#     # Same as day not know, but using for the doc string.
#     after_hear_day = hear(tell(parse_day(date)), prior_statement)
#     after_hear_month = hear(tell(parse_month(date)), prior_statement)
# #     print("mnk", len(after_hear_day) - len(after_hear_month))
#     if len(after_hear_day - after_hear_month) > 0:
#         return not know(after_hear_month)
#     return False


# # @cache
# def day_not_know(date, prior_statement):
#     "month: I still don't know, but I know that day does not know."
#     # Same as day not know, but using for the doc string.
#     after_hear_day = hear(tell(parse_day(date)), prior_statement)
#     after_hear_month = hear(tell(parse_month(date)), prior_statement)
#     if len(after_hear_month - after_hear_day) > 0:
#         return not know(after_hear_day)
#     return False

# # @cache
# def month_not_know(date, prior_statement):
#     "month: I still don't know, but I know that day does not know."
#     # Same as day not know, but using for the doc string.
#     after_hear_day = hear(tell(parse_day(date)), prior_statement)
#     after_hear_month = hear(tell(parse_month(date)), prior_statement)
# #     print("mnk", len(after_hear_day) - len(after_hear_month))
#     if len(after_hear_day - after_hear_month) > 0:
#         return not know(after_hear_month)
#     return False

def day_know(date, prior_statement):
    "day: Now I know when Cheryl's birthday is."
    return know(hear(tell(parse_day(date)), prior_statement))

def month_know(date, prior_statement):
    "Month: Now I also know when Cheryl's birthday is."
    return know(hear(tell(parse_month(date)), prior_statement))


statement_list = [month_initial]
statement_list.append(partial(day_not_know, prior_statement=statement_list[-1]))
statement_list.append(partial(month_not_know, prior_statement=statement_list[-1]))
statement_list.append(partial(day_not_know, prior_statement=statement_list[-1]))
statement_list.append(partial(month_know, prior_statement=statement_list[-1]))
statement_list.append(partial(day_know, prior_statement=statement_list[-1]))

statement_list_complicated = [month_initial]
statement_list_complicated.append(partial(day_not_know_complicated, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(month_not_know_complicated, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(day_not_know_complicated, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(month_know, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(day_know, prior_statement=statement_list_complicated[-1]))

# for i in range(2):
#     newstate = partial(day_not_know, prior_statement=statement_list[-1])
#     newstate.__doc__ = newstate.func.__doc__
#     print("Day -----", newstate)
#     statement_list.append(newstate)
#     newstate = partial(month_not_know, prior_statement=statement_list[-1])
#     newstate.__doc__ = newstate.func.__doc__
#     print("month -----", newstate)
#     statement_list.append(newstate)
# statement_list.append(partial(day_know, prior_statement=statement_list[-1]))
# statement_list.append(partial(month_know, prior_statement=statement_list[-1]))
# print(hear(dates, *statement_list[:4]))
# print(hear(dates, *statement_list_complicated[:4]))


In [10]:
statements = [albert1, bernard1, albert2, bernard2, albert3, bernard3]
statement_list = [month_initial]
statement_list.append(partial(day_not_know, prior_statement=statement_list[-1]))
statement_list.append(partial(month_not_know, prior_statement=statement_list[-1]))
statement_list.append(partial(day_not_know, prior_statement=statement_list[-1]))
statement_list.append(partial(month_know, prior_statement=statement_list[-1]))
statement_list.append(partial(day_know, prior_statement=statement_list[-1]))

statement_list_complicated = [month_initial]
statement_list_complicated.append(partial(day_not_know_complicated, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(month_not_know_complicated, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(day_not_know_complicated, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(month_know, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(day_know, prior_statement=statement_list_complicated[-1]))

dates, solution = gen_solution(statement_list, k=12, max_attempts=10000)
depth_check = 6
if dates != None:
    print(dates)
    print("--------------------")
    print(hear(dates, *statements[:depth_check]))
    print(hear(dates, *statement_list[:depth_check]))
    print(hear(dates, *statement_list_complicated[:depth_check]))
#     show_progression(dates, statements)


{'March 25', 'June 17', 'April 14', 'May 25', 'March 14', 'July 14', 'March 16', 'June 27', 'April 27', 'July 16', 'May 17', 'May 28'}
--------------------
{'April 14'}
{'April 14'}
{'April 14'}


In [48]:
def gen_dates_all(k=10):
    "Pick a set of dates"
    some_dates = {mo + ' ' + d1 + d2
              for mo in MONTHS[0:4]
              for d1 in '12'
              for d2 in '3456789'}

    return set(random.sample(tuple(some_dates), k))

def gen_solution_check_early(statements, k=10, max_attempts=10000):
    for _ in tqdm(range(max_attempts)):
        dates = gen_dates_all(k)
        using(dates)
        num_dates = len(dates)
        for i in range(1, len(statements) + 1):
            partial_solution = hear(dates, *statements[:i])
            # break if no solution or 
            # the number of possible isn't decreasing
            if partial_solution:
                if (i == len(statements)) and know(partial_solution):
                    return dates, partial_solution
                if len(partial_solution) < num_dates:
                    num_dates = len(partial_solution)
                else:
                    break
            else:
                break
    return None, None

statement_deep = [month_initial_simple]
for i in range(1):
    newstate = partial(day_not_know, prior_statement=statement_deep[-1])
    newstate.__doc__ = newstate.func.__doc__
    statement_deep.append(newstate)
    newstate = partial(month_not_know, prior_statement=statement_deep[-1])
    newstate.__doc__ = newstate.func.__doc__
    statement_deep.append(newstate)
statement_deep.pop(-1)
statement_deep.append(partial(month_know, prior_statement=statement_deep[-1]))
statement_deep.append(partial(day_know, prior_statement=statement_deep[-1]))

# statement_deep.append(partial(day_know, prior_statement=statement_deep[-1]))
# statement_deep.append(partial(month_know, prior_statement=statement_deep[-1]))

dates, solution = gen_solution_check_early(statement_deep, k=7, max_attempts=400000)
if dates != None:
    print(solution)
    show_progression(dates, statement_deep)
    


{'March 13'}





month: I don't know when Cheryl's birthday is, but I know that month does not know too.





Day: That helps, but I still don't know.





Month: Now I also know when Cheryl's birthday is.





day: Now I also know when Cheryl's birthday is.


In [50]:
good_dates = set()
current_day = 10
for m in MONTHS[:-1]:
    good_dates.add(m + " " + str(current_day))
    good_dates.add(m + " " + str(current_day + 1))
    current_day += 1
    
good_dates.add(MONTHS[1] + " " + str(10))
good_dates.add(MONTHS[-1] + " " + str(current_day))
    
show(good_dates)

In [54]:
statement_deep = [month_initial_simple]
for i in range(9):
    newstate = partial(day_not_know, prior_statement=statement_deep[-1])
    newstate.__doc__ = newstate.func.__doc__
    statement_deep.append(newstate)
    newstate = partial(month_not_know, prior_statement=statement_deep[-1])
    newstate.__doc__ = newstate.func.__doc__
    statement_deep.append(newstate)
statement_deep.append(partial(day_know, prior_statement=statement_deep[-1]))
statement_deep.append(partial(month_know, prior_statement=statement_deep[-1]))

show_progression(good_dates, statement_deep)




month: I don't know when Cheryl's birthday is.





Day: That helps, but I still don't know.





month: That helps, but I still don't know.





Day: That helps, but I still don't know.





month: That helps, but I still don't know.


...




Day: That helps, but I still don't know.





month: That helps, but I still don't know.





day: Now I know when Cheryl's birthday is.





Month: Now I also know when Cheryl's birthday is.


{'Febuary 12'}

In [229]:
def albert1(date):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_being_told = tell(parse_month(date))
    return (not know(after_being_told) 
            and all(not know(tell(parse_day(d)))
                    for d in after_being_told))

def bernard1(date):
    "Bernard: I still don't know"
    return not know(hear(tell(parse_day(date)), albert1))

def albert2(date):
    "Albert: I still don't know."
    return not know(hear(tell(parse_month(date)), bernard1))

def bernard2(date):
    "Bernard: I still don't know 2"
    return not know(hear(tell(parse_day(date)), albert2))

def bernard3(date):
    "Bernard: OK, now I know."
    return know(hear(tell(parse_day(date)), albert3))

def albert3(date):
    "Albert: OK, now I know."
    return know(hear(tell(parse_month(date)), bernard2))

statements = [albert1, bernard1, albert2, bernard2, albert3, bernard3]

In [437]:
dates = {'May 19', 'May 24', 'March 13', 'July 24', 'July 13', 'June 18', 'April 14', 'June 19', 'March 14', 'April 13'}
using(dates)
print(hear(dates, *statements[:6]))
print(hear(dates, *statement_list[:6]))

# print(gen_solution(statement_list, k=20, max_attempts=10000))
# show_progression(dates, statements)
# show_progression(dates, statements)


{'July 13'}
{'July 13'}


In [239]:
gabe_dates = [
  'January 15', 'January 4',
  'July 13',    'July 24',   'July 30',
  'March 13',   'March 24',
  'May 11',     'May 17',    'May 30']

cheryl_dates = DATES = {
     'May 15',    'May 16',    'May 19',
    'June 17',   'June 18',
    'July 14',   'July 16',
  'August 14', 'August 15', 'August 17'}

In [240]:
# Albert and Bernard just became friends with Cheryl, and they want to know when her birthday is. 
# Cheryl gives them a set of 10 possible dates:

def month(date): return date.split()[0]

def day(date):   return date.split()[1]

# Cheryl then tells Albert and Bernard separately 
# the month and the day of the birthday respectively.

def tell(part):
    "Cheryl tells a part of her birthdate; return a subset of DATES that match the part."
    return {date for date in DATES if part in date}

def know(possible_dates):
    "A person knows the birthdate if they know there is exactly one possible date."
    return len(possible_dates) == 1

def hear(possible_dates, *statements):
    "Return the subset of possible dates that are consistent with all the statements."
    return {date for date in possible_dates
            if all(stmt(date) for stmt in statements)}

# Albert and Bernard make three statements:

def albert1(date):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_being_told = tell(month(date))
    return (not know(after_being_told) 
            and all(not know(tell(day(d)))
                    for d in after_being_told))

def bernard1(date):
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    at_first = tell(day(date))
    return (not know(at_first)
            and know(hear(at_first, albert1)))


def albert2(date):
    "Albert: Then I also know when Cheryl's birthday is."
    return know(hear(tell(month(date)), bernard1))
    
# So when is Cheryl's birthday?

def cheryls_birthday(dates):
    "Return a list of the possible dates after hearing the three statements."
    return hear(using(dates), albert1, bernard1, albert2)

def using(dates):
    "Make dates be the value of the global variable DATES."
    global DATES # This is necessary because `tell` looks at `DATES`
    DATES = dates
    return dates

# Some tests

assert month('May 19') == 'May'
assert day('May 19') == '19'
assert albert1('May 19') == False
assert albert1('July 14') == True
assert know(tell('17')) == False
assert know(tell('19')) == True

In [241]:
print(cheryls_birthday(gabe_dates))
print(cheryls_birthday(cheryl_dates))

{'July 30'}
{'July 16'}


In [242]:
some_dates = {mo + ' ' + d1 + d2
              for mo in ('March', 'April', 'May', 'June', 'July')
              for d1 in '12'
              for d2 in '3456789'}

import random

def pick_dates(puzzle=cheryls_birthday, k=10):
    "Pick a set of dates for which the puzzle has a unique solution."
    while True:
        dates = random.sample(tuple(some_dates), k)
        solutions = puzzle(dates)
        if know(solutions):
            return solutions.pop(), dates

In [243]:
pick_dates()

('June 28', ['July 29', 'June 18', 'March 28', 'June 28', 'April 24', 'April 23', 'July 25', 'April 18', 'May 24', 'July 23'])

In [40]:
def albert1(date):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_being_told = tell(month(date))
    return (not know(after_being_told) 
            and all(not know(tell(day(d)))
                    for d in after_being_told))

def bernard1(date):
    "Bernard: I still don't know"
    return not know(hear(tell(day(date)), albert1))

def albert2(date):
    "Albert: I still don't know."
    return not know(hear(tell(month(date)), bernard1))

def bernard2(date):
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    return not know(hear(tell(day(date)), albert2))

def albert3(date):
    "Albert: OK, now I know."
    return know(hear(tell(month(date)), bernard2))

def bernard3(date):
    "Bernard: OK, now I know."
    return know(hear(tell(day(date)), albert3))

In [41]:
def cheryls_birthday_complex(dates):
    "Return a set of the dates for which Albert, Bernard, and Eve's statements are true."
    return hear(using(dates), albert1, bernard1, albert2, bernard2, albert3, bernard3)

In [42]:
output = pick_dates(puzzle=cheryls_birthday_complex)
result, dates = output[0], output[1]
print(result)
print(dates)

July 13
['May 19', 'May 24', 'March 13', 'July 24', 'July 13', 'June 18', 'April 14', 'June 19', 'March 14', 'April 13']


In [43]:
dates = ['May 19', 'May 24', 'March 13', 'July 24', 'July 13', 'June 18', 'April 14', 'June 19', 'March 14', 'April 13']

In [45]:
hear(dates, albert1)

{'April 13',
 'April 14',
 'July 13',
 'July 24',
 'March 13',
 'March 14',
 'May 19',
 'May 24'}

In [46]:
hear(dates, albert1, bernard1)

{'April 13',
 'April 14',
 'July 13',
 'July 24',
 'March 13',
 'March 14',
 'May 24'}

In [47]:
hear(dates, albert1, bernard1, albert2)

{'April 13', 'April 14', 'July 13', 'July 24', 'March 13', 'March 14'}

In [48]:
hear(dates, albert1, bernard1, albert2, bernard2)

{'April 13', 'April 14', 'July 13', 'March 13', 'March 14'}

In [49]:
hear(dates, albert1, bernard1, albert2, bernard2, albert3)

{'July 13'}

{'June 26', 'June 13', 'April 19', 'May 14', 'March 27', 'July 16', 'March 25', 'June 24', 'April 24', 'May 27', 'June 25', 'April 29', 'March 26', 'July 14', 'March 24'}

In [321]:
def a(b, x = 1):
    """in a"""
    return b * x

val = partial(a, x = 10)

In [338]:
val.func.__doc__

'in a'

In [396]:
def tell(part, possible_dates):
    "Cheryl tells a part of her birthdate; return a subset of possible_dates that match the part."
    return {date for date in possible_dates if part in date}

def know(possible_dates):
    "A person knows the birthdate if they know there is exactly one possible date."
    return len(possible_dates) == 1

def hear(possible_dates, statements):
    "Return the subset of possible dates that are consistent with all the statements."
    current_possible = possible_dates
    for stmt in statements:
        print(stmt)
        tmp_possible = set()
        for date in current_possible:
            print(date)
            if stmt(date, possible_dates):
                tmp_possible.add(date)
        current_possible = tmp_possible
        print(current_possible)
    return current_possible

def show_with_statements(dates, statements):
    return_set = hear(dates, statements)
    show(dates, dates - 
         hear(dates, statements))
    return return_set

# def show_progression(dates, statements):
#     using(dates)
#     for i in range(len(statements) + 1):
#         if i > 0: 
#             print("\n\n")
#             if type(statements[i-1]) is partial:
#                 print(statements[i-1].func.__doc__)
#             else:
#                 print(statements[i-1].__doc__)
#         return_set = show_with_statements(dates, statements[:i])
#     return return_set


#Statments: For a given date - returns true / false if that date is possibly a birthday
def albert1(date, possible_dates):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_told = tell(parse_month(date), possible_dates)
    return (not know(after_told) 
            and all(not know(tell(parse_day(d), possible_dates))
                    for d in after_told))

def bernard1(date, possible_dates):
    "Bernard: At first I don't know when Cheryl's birthday is, but I know now."
    at_first = tell(parse_day(date), possible_dates)
    print("b:", at_first)
    return (not know(at_first)
            and know(hear(at_first, [albert1])))

def albert2(date, possible_dates):
    "Albert: Then I also know when Cheryl's birthday is."
    after_told = tell(parse_month(date), possible_dates)
    return know(hear(after_told, [bernard1]))


cheryl_dates = {
     'May 15',    'May 16',    'May 19',
    'June 17',   'June 18',
    'July 14',   'July 16',
  'August 14', 'August 15', 'August 17'}

statements = [albert1, bernard1, albert2]
hear(cheryl_dates, [albert1, bernard1])
# show_with_statements(dates, statements)

<function albert1 at 0x10b44d8b0>
July 16
May 19
August 14
June 17
May 15
May 16
June 18
August 15
August 17
July 14
{'July 16', 'August 17', 'August 14', 'August 15', 'July 14'}
<function bernard1 at 0x10ac6bca0>
July 16
b: {'July 16', 'May 16'}
<function albert1 at 0x10b44d8b0>
July 16
May 16
set()
August 17
b: {'August 17', 'June 17'}
<function albert1 at 0x10b44d8b0>
August 17
June 17
set()
August 14
b: {'July 14', 'August 14'}
<function albert1 at 0x10b44d8b0>
July 14
August 14
set()
August 15
b: {'August 15', 'May 15'}
<function albert1 at 0x10b44d8b0>
August 15
May 15
set()
July 14
b: {'July 14', 'August 14'}
<function albert1 at 0x10b44d8b0>
July 14
August 14
set()
set()


set()

# Interesting scenarios

In [152]:
dates = {'April 16',
 'April 24',
 'July 14',
 'July 18',
 'July 29',
 'June 15',
 'June 16',
 'June 25',
 'March 24',
 'March 29',
 'May 16',
 'May 18'}

def albert1(date):
    "Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too."
    after_being_told = tell(parse_month(date))
    return (not know(after_being_told) 
            and all(not know(tell(parse_day(d)))
                    for d in after_being_told))

def bernard1(date):
    "Bernard: I still don't know"
    return not know(hear(tell(parse_day(date)), albert1))

def albert2(date):
    "Albert: I still don't know."
    return not know(hear(tell(parse_month(date)), bernard1))

def bernard2(date):
    "Bernard: I still don't know 2"
    return not know(hear(tell(parse_day(date)), albert2))

def bernard3(date):
    "Bernard: OK, now I know."
    return know(hear(tell(parse_day(date)), albert3))

def albert3(date):
    "Albert: OK, now I know."
    return know(hear(tell(parse_month(date)), bernard2))


def month_initial(date):
    "month: I don't know when Cheryl's birthday is, but I know that month does not know too."
    after_being_told = tell(parse_month(date))
    return (not know(after_being_told) 
            and all(not know(tell(parse_day(d)))
                    for d in after_being_told))

def day_not_know_complicated(date, prior_statement):
    "day: I still don't know, but I know that month does not know."
    after_hear_day = hear(tell(parse_day(date)), prior_statement)
    after_hear_month = hear(tell(parse_month(date)), prior_statement)
    return (not know(after_hear_day)
            and all(not know(after_hear_month)
                    for d in after_hear_day))

def month_not_know_complicated(date, prior_statement):
    "month: I still don't know, but I know that day does not know."
    after_hear_day = hear(tell(parse_day(date)), prior_statement)
    after_hear_month = hear(tell(parse_month(date)), prior_statement)
    return (not know(after_hear_month)
            and all(not know(after_hear_day)
                    for d in after_hear_month))

# @cache
def day_not_know(date, prior_statement):
    "month: I still don't know, but I know that day does not know."
    # Same as day not know, but using for the doc string.
    after_hear_day = hear(tell(parse_day(date)), prior_statement)
    after_hear_month = hear(tell(parse_month(date)), prior_statement)
    if len(after_hear_month - after_hear_day) > 0:
        return not know(after_hear_day)
    return False

# @cache
def month_not_know(date, prior_statement):
    "month: I still don't know, but I know that day does not know."
    # Same as day not know, but using for the doc string.
    after_hear_day = hear(tell(parse_day(date)), prior_statement)
    after_hear_month = hear(tell(parse_month(date)), prior_statement)
#     print("mnk", len(after_hear_day) - len(after_hear_month))
    if len(after_hear_day - after_hear_month) > 0:
        return not know(after_hear_month)
    return False

def day_know(date, prior_statement):
    "day: Now I also know when Cheryl's birthday is."
    return know(hear(tell(parse_day(date)), prior_statement))

def month_know(date, prior_statement):
    "Month: Now I also know when Cheryl's birthday is."
    return know(hear(tell(parse_month(date)), prior_statement))


statement_list = [month_initial]
statement_list.append(partial(day_not_know, prior_statement=statement_list[-1]))
statement_list.append(partial(month_not_know, prior_statement=statement_list[-1]))
statement_list.append(partial(day_not_know, prior_statement=statement_list[-1]))
statement_list.append(partial(month_know, prior_statement=statement_list[-1]))
statement_list.append(partial(day_know, prior_statement=statement_list[-1]))

statement_list_complicated = [month_initial]
statement_list_complicated.append(partial(day_not_know_complicated, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(month_not_know_complicated, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(day_not_know_complicated, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(month_know, prior_statement=statement_list_complicated[-1]))
statement_list_complicated.append(partial(day_know, prior_statement=statement_list_complicated[-1]))

depth_check = 6
if dates != None:
    print(dates)
    print("--------------------")
    print(hear(dates, *statements[:depth_check]))
    print(hear(dates, *statement_list[:depth_check]))
    print(hear(dates, *statement_list_complicated[:depth_check]))
    show_progression(dates, statements)


{'June 15', 'March 24', 'May 18', 'July 18', 'June 16', 'April 24', 'April 16', 'March 29', 'July 14', 'June 25', 'May 16', 'July 29'}
--------------------
set()
{'April 16'}
{'April 16'}





Albert: I don't know when Cheryl's birthday is, but I know that Bernard does not know too.





Bernard: I still don't know





Albert: I still don't know.





Bernard: I still don't know 2





Albert: OK, now I know.





Bernard: OK, now I know.
